# Useful Alyx REST queries
Examples of using ONE to query Alyx via Django REST queries.

Alyx queries require and internet connection and are slower than the local cache queries using
one.search, however it is much more powerful.  Searching for datasets or sessions based solely on
the fields in `one.search_terms` should be done using one.search.  This script demonstrates some
of the more complex queries that must be done remotely.

In [9]:
from pprint import pprint
from one.api import ONE, OneAlyx

one = ONE(base_url='https://openalyx.internationalbrainlab.org')
assert isinstance(one, OneAlyx) and not one.offline

## Exploring the REST endpoints
Full documentation of using Alyx REST interface can be found at
https://openalyx.internationalbrainlab.org/docs

This URL is itself a REST endpoint, so you can list the endpoints through ONE:

In [10]:
print(one.alyx.list_endpoints())  # Can also be done with `one.alyx.rest()`

['brain-regions', 'cache', 'cache.zip', 'channels', 'data-formats', 'data-repository', 'data-repository-type', 'dataset-types', 'datasets', 'downloads', 'files', 'insertions', 'labs', 'locations', 'new-download', 'notes', 'projects', 'register-file', 'revisions', 'sessions', 'subjects', 'surgeries', 'sync-file-status', 'tags', 'tasks', 'trajectories', 'uploaded', 'users', 'water-administrations', 'water-requirement', 'water-restricted-subjects', 'water-restriction', 'water-type', 'weighings']


The main GET requests are 'list' and 'read'.  The parameters for each are described in the
`rest_schemas` property.  For example, for the parameters available for listing sessions...

In [11]:
print(one.alyx.rest_schemes['sessions']['list']['description'])

**FILTERS**

-   **subject**: subject nickname `/sessions?subject=Algernon`
-   **dataset_types**: dataset type
-   **number**: session number
-   **users**: experimenters (exact)
-   **date_range**: date `/sessions?date_range=2020-01-12,2020-01-16`
-   **lab**: lab name (exact)
-   **task_protocol** (icontains)
-   **location**: location name (icontains)
-   **project**: project name (icontains)
-   **json**: queries on json fields, for example here `tutu`
    -   exact/equal lookup: `/sessions?extended_qc=tutu,True`,
    -   gte lookup: `/sessions/?extended_qc=tutu__gte,0.5`,
-   **extended_qc** queries on json fields, for example here `qc_bool` and `qc_pct`,
    values and fields come by pairs, using semi-colon as a separator
    -   exact/equal lookup: `/sessions?extended_qc=qc_bool;True`,
    -   gte lookup: `/sessions/?extended_qc=qc_pct__gte;0.5`,
    -   chained lookups: `/sessions/?extended_qc=qc_pct__gte;0.5;qc_bool;True`,
-   **performance_gte**, **performance_lte**: percent

## Example queries
### convert session dicts to eids
The output of the below queries can be easily converted to eID strings with the following line:
```python
eids = one.to_eid(ses)
```

### list sessions that have histology available
```python
# The sessions endpoint has a `histology` parameter:
ses = one.alyx.rest('sessions', 'list', histology=True)

# The generic way is to use the `django` parameter:
ses = one.alyx.rest('sessions', 'list',
                    django='subject__actions_sessions__procedures__name,Histology')
```

### list sessions having specific channel locations (hierarchical, will fetch everything below)
```python
ses = one.alyx.rest('sessions', 'list', atlas_id=500)
ses = one.alyx.rest('sessions', 'list', atlas_acronym="MO")
ses = one.alyx.rest('sessions', 'list', atlas_name="Somatomotor areas")

```
### list sessions that do not have matlab in the project name
```python
ses = one.alyx.rest('sessions', 'list', django='~project__name__icontains,matlab')
```

### list trajectories with channels in given brain region
```python
trajs = one.alyx.rest('trajectories', 'list', django='channels__brain_region__name__icontains,'
                                                     'Entorhinal area medial part dorsal zone '
                                                     'layer 2')
```

### list insertions that have alignment resolved
```python
ins = one.alyx.rest('insertions', 'list', django='json__extended_qc__alignment_resolved,True')
```

### list names of users who have aligned specified insertion
```python
insertion_uuid = 'b749446c-18e3-4987-820a-50649ab0f826'
traj = one.alyx.rest('trajectories', 'list',
                      provenance='Ephys aligned histology track',
                      probe_insertion=insertion_uuid)
names = traj[0]['json'].keys()
```

### list probe insertions for a given task protocol
```python
ins = one.alyx.rest('insertions', 'list', django='session__task_protocol__icontains,choiceworld')
```

### list spikesorting tasks that have errored in a given lab
```python
errored = one.alyx.rest('tasks', 'list', status='Errored', lab='angelakilab', name='SpikeSorting')
```

### list ephys sessions that have errored tasks
```python
ses = one.alyx.rest('sessions', 'list', task_protocol='ephys', django='tasks__status,40')
```

### rerun / set errored tasks to Waiting
```python
# List tasks with given status
tasks = one.alyx.rest('tasks', 'list', name='EphysDLC', status='Errored')

# OR tasks with specific log error message
tasks = one.alyx.rest('tasks', 'list', name='EphysDLC',
                      django="log__icontains,TimeoutError: [Errno 110]")
# Set tasks to Waiting so they get rerun
for t in tasks:
    one.alyx.rest('tasks', 'partial_update', id=t['id'], data={'log': "", 'status': 'Waiting'}
```

### list sessions where extended QC exists for any video
```python
keys = ('videoLeft', 'videoRight', 'videoBody')
one.alyx.rest('sessions', 'list', django=f'extended_qc__has_any_keys,{keys}')
```

In [ ]:
## Field lookup reference
With the `django` parameter you can write custom filters.  The syntax is `'field1__lookup,query,
field2__lookup,query'` (the field and lookup query are separated by two underscores).  For example
`'nickname__icontains,ks,death_date__isnull,True'`.  Multiple lookups are separated by commas,
forming a logical AND (it is not possible to construct OR queries, instead make separate queries).

The lookups translate to a [SQL WHERE clause](https://www.w3schools.com/SQl/sql_where.asp).

### Related field lookups
Some fields are actually related tables, whose fields can also used in the filter.  For example,
the `subject` field of the `sessions` table contains all the subject table fields.  Lookups can
be applied to these fields in the same way: `field__subfield__lookup,query` e.g.
`sessions__subject__nickname__icontains,dop` ('find sessions where the subject's name contains
"dop"').

See the django [QuerySet API documentation](https://docs.djangoproject.com/en/3.2/ref/models/querysets/#field-lookups)
and [PostgreSQL specific documentation](https://docs.djangoproject.com/en/3.2/ref/contrib/postgres/fields/) for more details.

### JSON field lookups

JSON fields can be filtered in the same way as related tables.  For example, the `sessions`
table contains a 'extended_qc' JSON field that contains a map of QC checks and their outcomes.
The fields and values can be used in lookup queries.  For more info, see [Querying JSONField](https://docs.djangoproject.com/en/3.2/topics/db/queries/#querying-jsonfield).

Here's how the extended_qc field looks:

ses = one.alyx.rest('sessions', 'read', id='4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a')
pprint(ses['extended_qc'])

#### Looking up fields
```python
# Find sessions where task QC is marked as 'FAIL'
one.alyx.rest('sessions', 'list', django='extended_qc__task__iexact,fail')
```

Lists can be accessed using indices:
```python
# Find sessions where first value (zero-index) of '_videoLeft_pin_state' is greater than 5
one.alyx.rest('sessions', 'list', django='extended_qc___videoLeft_pin_state__0__gt,5')
```

Any search depth is allowed, and lookups such as `isnull`, `startswith`, `gte`, etc. are permitted.

#### contains
The returned objects are those where the given dict of key-value pairs are all contained in the
top-level of the field.
```python
# Find sessions where extended QC has 'dlcLeft' and 'videoLeft' both pass:
d = {'dlcLeft': 'PASS', 'videoLeft': 'PASS'}
one.alyx.rest('sessions', 'list', django=f'extended_qc__contains,{d}')
```

#### contained_by
This is the inverse of the contains lookup - the objects returned will be those where the key-value
pairs on the object are a subset of those in the value passed.
```python
# Find sessions where extended QC has 'dlcLeft' and 'videoLeft' are missing or do not pass:
d = {'dlcLeft': 'PASS', 'videoLeft': 'PASS'}
one.alyx.rest('sessions', 'list', django=f'extended_qc__contained_by,{d}')
```

#### has_key
JSON field contains a given key.
```python
# Find sessions where extended QC has 'dlcLeft' field:
one.alyx.rest('sessions', 'list', django='extended_qc__has_key,dlcLeft')
```

#### has_keys
JSON contains all of the listed keys.  The list should be surrounded by parentheses or
square brackets, e.g. `'field__has_keys,['field1', 'field2']'` or `'field__has_keys,('field1', 'field2')'`
```python
# Find sessions where extended QC 'behavior' and 'ephys' fields:
keys = ['behavior', 'ephys']
one.alyx.rest('sessions', 'list', django=f'extended_qc__has_keys,{keys}')
```

#### has_any_keys
JSON contains at least one of the listed keys.  The list should be surrounded by parentheses or
square brackets, e.g. `'field__has_any_keys,['field1', 'field2']'` or `'field__has_any_keys,('field1', 'field2')'`
```python
# Find sessions where extended QC exists for any video
keys = ('videoLeft', 'videoRight', 'videoBody')
one.alyx.rest('sessions', 'list', django=f'extended_qc__has_any_keys,{keys}')
```

### exact
An exact match.  When the lookup is omitted it is assumed to be `exact`:
```python
one.alyx.rest('sessions', 'list', django='subject__nickname__exact,KS022')
one.alyx.rest('sessions', 'list', django='subject__nickname,KS022')  # equivalent
```

### iexact
Case insensitive exact match:
```python
one.alyx.rest('sessions', 'list', django='subject__nickname__exact,ks022')
```

### contains
### icontains
Search for records where a given field contains a substring, case insensitive.  For examples to
query insertions with a task protocol containing 'choiceworld':
```python
ins = one.alyx.rest('insertions', 'list', django='session__task_protocol__icontains,choiceworld')
```

### in
In a given iterable; often a list, tuple, or queryset. It’s not a common use case, but strings
(being iterables) are accepted.  For example to query session for several subjects:
```python
subjects = ['DY_003', 'DY_006']
ses = one.alyx.rest('sessions', 'list', django=f"subject__nickname__in,{subjects}")
```
```{note}
This example query can be done more efficiently with `one.search(subject=['DY_003', 'DY_006'])`
```

### gt
### gte
### lt
Less than.  Works for datetime, date and numerical fields:
```python
one.alyx.rest('sessions', 'list', django='session__qc__lt,40')  # Where QC less than 40
```
### lte
Less than or equal.  Works for datetime, date and numerical fields.  For example to get
insertions for sessions on or before a given date:
```python
one.alyx.rest('insertions', 'list', django='session__start_time__date__lte,2021-07-22')
```

### startswith
### istartswith
### endswith
### iendswith

### not
A tilda can be used to negate any query filter, for example, to query sessions that **do not**
have matlab in the project name:
```python
ses = one.alyx.rest('sessions', 'list', django='~project__name__icontains,matlab')
```

### range

### date
Extract the date part from a date_time field.  For example to search for a given date:
```python
date = '2021-02-05'
weighings = one.alyx.rest('weighings', 'list', django=f'date_time__date,{date}')
```

### year
Extract the year from a date_time field.

### iso_year
### month
### day
### week
### week_day
### iso_week_day
### quarter
### time
### hour
### minute
### second

### isnull
### regex
Case-sensitive regular expression match.  The regular expressions should be supported by
PostgreSQL.  More info on the syntax [here](https://www.postgresql.org/docs/9.3/functions-matching.html).
```python
# For a given session, find datasets that belong to a probe collection
eid = '4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a'
one.alyx.rest('datasets', 'list', session=eid, django='collection__regex,.*probe.*', exists=True)
```
```{note}
This example query can be done more efficiently with `one.list(eid, collection='*probe*')`
```

### iregex
Case-insensitive regular expression match.  The regular expressions should be supported by
PostgreSQL.  More info on the syntax [here](https://www.postgresql.org/docs/9.3/functions-matching.html).

```python
# List datasets for a given session that are related to either the right or left videos
eid = '4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a'
query = 'name__iregex,(right|left)camera'
one.alyx.rest('datasets', 'list', session=eid, django=query, exists=True)
```
```{note}
This example query can be done more efficiently with `one.list_datasets(eid, filename={'object': ['leftCamera', 'rightCamera']})`
```

#### Looking up fields
```python
# Find sessions where task QC is marked as 'FAIL'
one.alyx.rest('sessions', 'list', django='extended_qc__task__iexact,fail')
```

Lists can be accessed using indices:
```python
# Find sessions where first value (zero-index) of '_videoLeft_pin_state' is greater than 5
one.alyx.rest('sessions', 'list', django='extended_qc___videoLeft_pin_state__0__gt,5')
```

Any search depth is allowed, and lookups such as `isnull`, `startswith`, `gte`, etc. are permitted.

#### contains
The returned objects are those where the given dict of key-value pairs are all contained in the
top-level of the field.
```python
# Find sessions where extended QC has 'dlcLeft' and 'videoLeft' both pass:
d = {'dlcLeft': 'PASS', 'videoLeft': 'PASS'}
one.alyx.rest('sessions', 'list', django=f'extended_qc__contains,{d}')
```

#### contained_by
This is the inverse of the contains lookup - the objects returned will be those where the key-value
pairs on the object are a subset of those in the value passed.
```python
# Find sessions where extended QC has 'dlcLeft' and 'videoLeft' are missing or do not pass:
d = {'dlcLeft': 'PASS', 'videoLeft': 'PASS'}
one.alyx.rest('sessions', 'list', django=f'extended_qc__contained_by,{d}')
```

#### has_key
JSON field contains a given key.
```python
# Find sessions where extended QC has 'dlcLeft' field:
one.alyx.rest('sessions', 'list', django='extended_qc__has_key,dlcLeft')
```

#### has_keys
JSON contains all of the listed keys.  The list should be surrounded by parentheses or
square brackets, e.g. `'field__has_keys,['field1', 'field2']'` or `'field__has_keys,('field1', 'field2')'`
```python
# Find sessions where extended QC 'behavior' and 'ephys' fields:
keys = ['behavior', 'ephys']
one.alyx.rest('sessions', 'list', django=f'extended_qc__has_keys,{keys}')
```

#### has_any_keys
JSON contains at least one of the listed keys.  The list should be surrounded by parentheses or
square brackets, e.g. `'field__has_any_keys,['field1', 'field2']'` or `'field__has_any_keys,('field1', 'field2')'`
```python
# Find sessions where extended QC exists for any video
keys = ('videoLeft', 'videoRight', 'videoBody')
one.alyx.rest('sessions', 'list', django=f'extended_qc__has_any_keys,{keys}')
```

### exact
An exact match.  When the lookup is omitted it is assumed to be `exact`:
```python
one.alyx.rest('sessions', 'list', django='subject__nickname__exact,KS022')
one.alyx.rest('sessions', 'list', django='subject__nickname,KS022')  # equivalent
```

### iexact
Case insensitive exact match:
```python
one.alyx.rest('sessions', 'list', django='subject__nickname__exact,ks022')
```

### contains
### icontains
Search for records where a given field contains a substring, case insensitive.  For examples to
query insertions with a task protocol containing 'choiceworld':
```python
ins = one.alyx.rest('insertions', 'list', django='session__task_protocol__icontains,choiceworld')
```

### in
In a given iterable; often a list, tuple, or queryset. It’s not a common use case, but strings
(being iterables) are accepted.  For example to query session for several subjects:
```python
subjects = ['DY_003', 'DY_006']
ses = one.alyx.rest('sessions', 'list', django=f"subject__nickname__in,{subjects}")
```
```{note}
This example query can be done more efficiently with `one.search(subject=['DY_003', 'DY_006'])`
```

### gt
### gte
### lt
Less than.  Works for datetime, date and numerical fields:
```python
one.alyx.rest('sessions', 'list', django='session__qc__lt,40')  # Where QC less than 40
```
### lte
### startswith
### istartswith
### endswith
### iendswith

### not
A tilda can be used to negate any query filter, for example, to query sessions that **do not**
have matlab in the project name:
```python
ses = one.alyx.rest('sessions', 'list', django='~project__name__icontains,matlab')
```

### range

### date
Get the date from a datetime field:
```python
one.alyx.rest('insertions', 'list', django='session__start_time__date__lt,2021-07-22')
```

### year
### iso_year
### month
### day
### week
### week_day
### iso_week_day
### quarter
### time
### hour
### minute
### second

### isnull
### regex
Case-sensitive regular expression match.  The regular expressions should be supported by
PostgreSQL.  More info on the syntax [here](https://www.postgresql.org/docs/9.3/functions-matching.html).
```python
# For a given session, find datasets that belong to a probe collection
eid = '4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a'
one.alyx.rest('datasets', 'list', session=eid, django='collection__regex,.*probe.*', exists=True)
```
```{note}
This example query can be done more efficiently with `one.list(eid, collection='*probe*')`
```

### iregex
Case-insensitive regular expression match.  The regular expressions should be supported by
PostgreSQL.  More info on the syntax [here](https://www.postgresql.org/docs/9.3/functions-matching.html).

```python
# List datasets for a given session that are related to either the right or left videos
eid = '4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a'
query = 'name__iregex,(right|left)camera'
one.alyx.rest('datasets', 'list', session=eid, django=query, exists=True)
```
```{note}
This example query can be done more efficiently with `one.list_datasets(eid, filename={'object': ['leftCamera', 'rightCamera']})`
```